In [1]:
# !pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [12]:
driver = webdriver.Chrome()
# Islamabad Page
driver.get('https://www.zameen.com/new-projects/islamabad-3-1/')


# Get all the anchor tags
all_projects_selector = 'main > div > div > div > div > div > a'

elements = driver.find_elements(By.CSS_SELECTOR, all_projects_selector)


In [14]:
for idx, ele in enumerate(elements):
    print(idx, ele.get_attribute('outerHTML'))

0 <a href="/new-projects/grand_orchard-2275/"><div class="card_card__3KEPg new-project-card_newProjectsCard__3d8AK" style="--padding:20px;--radius:10px;background-color:#fff"><div class="space_spaceFlex__35XH1 space_align-start__apUk4" style="gap:44px;flex-wrap:"><div class="new-project-card_thumbGallery__2RMfB undefined"><div class="new-project-card_share__2CNlK new-project-card_shareDropdown__FwUpC dropdown_dropdown__2rjLb"><button class="dropdown_dropdownTrigger__2vlKf dropdown_btnPill__3kUZX"><svg stroke="currentColor" fill="currentColor" stroke-width="0" viewBox="0 0 20 20" aria-hidden="true" height="1em" width="1em" xmlns="http://www.w3.org/2000/svg"><path d="M15 8a3 3 0 10-2.977-2.63l-4.94 2.47a3 3 0 100 4.319l4.94 2.47a3 3 0 10.895-1.789l-4.94-2.47a3.027 3.027 0 000-.74l4.94-2.47C13.456 7.68 14.19 8 15 8z"></path></svg> </button><div class="dropdown_dropdownMenu__1Tq2Y"><button aria-label="facebook" class="react-share__ShareButton social-share-menu_iconShare__2E4oz" style="back